# GTEx model with different priors

## Load libraries

In [2]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER2)

source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses



## Output directory

In [3]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

# GTEx model with different priors

In [4]:
options(repr.matrix.max.rows = Inf)

# List all saved models
rds_files <- list.files(output_data_dir, pattern = "^gtex_.*_PLIER2\\.rds$", full.names = TRUE)

summaries <- list()

for (file in rds_files) {
  prior_name <- sub("^gtex_(.*)_PLIER2\\.rds$", "\\1", basename(file))
  message("Reading summary for: ", prior_name)

  res <- readRDS(file)
  if (!is.null(res$summary)) {
    summaries[[prior_name]] <- res$summary %>%
      dplyr::filter(FDR < 0.05 & AUC > 0.7)
  }
}

# Combine into one data frame with prior name column
all_summaries <- dplyr::bind_rows(
  lapply(names(summaries), function(p) {
    dplyr::mutate(summaries[[p]], prior = p)
  })
)

Reading summary for: all



Reading summary for: Chromosome_Location

Reading summary for: GO_Biological_Process_2025

Reading summary for: GTEx_Tissues_V8_2023

Reading summary for: GWAS_Catalog_2025

Reading summary for: Human_Gene_Atlas

Reading summary for: KEGG_2021_Human

Reading summary for: LINCS_L1000_CRISPR_KO_Consensus_Sigs

Reading summary for: Metabolomics_Workbench_Metabolites_2022

Reading summary for: OMIM_Disease

Reading summary for: Proteomics_Drug_Atlas_2023

Reading summary for: TF_Perturbations_Followed_by_Expression

Reading summary for: UK_Biobank_GWAS_v1



In [5]:
all_summaries %>%
    dplyr::mutate(pathway_prior = paste0(prior, '_', pathway)) %>%
    dplyr::select(LV, pathway_prior) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway_prior = paste(pathway_prior, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

LV,pathway_prior
<chr>,<chr>
LV1,"GO_Biological_Process_2025_Calcium-Dependent Cell-Cell Adhesion via Plasma Membrane Cell Adhesion Molecules (GO:0016339), GTEx_Tissues_V8_2023_Artery - Tibial Male 60-69 Up, GTEx_Tissues_V8_2023_Artery - Aorta Female 60-69 Up, GTEx_Tissues_V8_2023_Artery - Aorta Female 20-29 Up, GTEx_Tissues_V8_2023_Artery - Coronary Male 30-39 Up, GTEx_Tissues_V8_2023_Artery - Aorta Female 40-49 Up, GTEx_Tissues_V8_2023_Artery - Aorta Female 70-79 Up"
LV2,"GTEx_Tissues_V8_2023_Brain - Cerebellum Male 50-59 Up, GTEx_Tissues_V8_2023_Brain - Cerebellar Hemisphere Male 30-39 Up, GTEx_Tissues_V8_2023_Brain - Cerebellum Female 50-59 Up, GTEx_Tissues_V8_2023_Brain - Cerebellar Hemisphere Male 70-79 Up, GTEx_Tissues_V8_2023_Brain - Cerebellar Hemisphere Female 40-49 Up, GTEx_Tissues_V8_2023_Brain - Cerebellar Hemisphere Female 50-59 Up, GTEx_Tissues_V8_2023_Brain - Cerebellar Hemisphere Female 20-29 Up, GTEx_Tissues_V8_2023_Brain - Cerebellum Female 30-39 Up, Human_Gene_Atlas_CerebellumPeduncles, Human_Gene_Atlas_Cerebellum"
LV3,"all_Chromosome_Location_chr2p11, all_Chromosome_Location_chr14q32, Chromosome_Location_chr14q32, GO_Biological_Process_2025_Antigen Receptor-Mediated Signaling Pathway (GO:0050851), GO_Biological_Process_2025_Regulation of B Cell Proliferation (GO:0030888), GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Female 60-69 Up, GTEx_Tissues_V8_2023_Minor Salivary Gland Male 50-59 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Female 20-29 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Male 30-39 Up, GTEx_Tissues_V8_2023_Colon - Transverse Male 40-49 Up, GTEx_Tissues_V8_2023_Spleen Female 30-39 Up, GTEx_Tissues_V8_2023_Spleen Female 50-59 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Male 40-49 Up, GTEx_Tissues_V8_2023_Small Intestine - Terminal Ileum Male 70-79 Up, GTEx_Tissues_V8_2023_Whole Blood Female 70-79 Up, Human_Gene_Atlas_colon, Human_Gene_Atlas_Thymus, Human_Gene_Atlas_Trachea, Human_Gene_Atlas_Lymphnode, KEGG_2021_Human_Autoimmune thyroid disease, KEGG_2021_Human_Cytokine-cytokine receptor interaction, KEGG_2021_Human_Hematopoietic cell lineage, KEGG_2021_Human_Intestinal immune network for IgA production, TF_Perturbations_Followed_by_Expression_FOSL1 KO MOUSE GSE43695 CREEDSID GENE 2303 DOWN"
LV4,"all_BP_Positive Regulation of Type II Interferon Production (GO:0032729), all_GTEx_Tissues_Cells - EBV-transformed Lymphocytes Male 30-39 Up, all_GTEx_Tissues_Cells - EBV-transformed Lymphocytes Male 20-29 Up, all_GTEx_Tissues_Cells - EBV-transformed Lymphocytes Female 40-49 Up, GO_Biological_Process_2025_Fc-gamma Receptor Signaling Pathway Involved in Phagocytosis (GO:0038096), GO_Biological_Process_2025_Regulation of B Cell Receptor Signaling Pathway (GO:0050855), GTEx_Tissues_V8_2023_Whole Blood Male 50-59 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Female 20-29 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Male 60-69 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Male 20-29 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Female 40-49 Up, GTEx_Tissues_V8_2023_Spleen Female 50-59 Up, GTEx_Tissues_V8_2023_Cells - EBV-transformed Lymphocytes Male 40-49 Up, KEGG_2021_Human_Allograft rejection, KEGG_2021_Human_Intestinal immune network for IgA production, TF_Perturbations_Followed_by_Expression_E2F8 KO MOUSE GSE52157 CREEDSID GENE 2828 DOWN"
LV5,"all_GTEx_Tissues_Muscle - Skeletal Male 60-69 Up, GO_Biological_Process_2025_Calcineurin-Mediated Signaling (GO:0097720), GTEx_Tissues_V8_2023_Heart - Left Ventricle Male 50-59 Up, GTEx_Tissues_V8_2023_Heart - Left Ventricle Male 20-29 Up, GTEx_Tissues_V8_2023_Heart - Left Ventricle Female 30-39 Up, Human_Gene_Atlas_Heart, Human_Gene_Atlas_Kidney, KEGG_2021_Human_Citrate cycle (TCA cycle), KEGG_2021_Human_Oxidative phosphorylation, KEGG_2021_Human_Valine, leucine and isoleucine degradation, Metabolomics_Workbench_Metabolites_2022_Coenzyme A, Pr

In [6]:
summ_all_summaries  <- all_summaries %>%
    dplyr::mutate(pathway_prior = paste0(prior, '_', pathway)) %>%
    dplyr::select(LV, pathway_prior) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway_prior = paste(pathway_prior, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

saveRDS(summ_all_summaries, file = file.path(output_data_dir, "all_summaries.rds"))

In [9]:
options(repr.matrix.max.rows = Inf)

gtex_fullRes_full <- readRDS(file.path(output_data_dir, "gtex_all_PLIER2.rds"))

gtex_fullRes_full_summ  <- gtex_fullRes_full$summary  %>% 
dplyr::filter(FDR < 0.05 & AUC > 0.7)  %>% 
dplyr::group_by(LV) %>% 
dplyr::summarise(pathway_prior = paste(pathway, collapse = ', '), .groups = "drop") %>%
dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

gtex_fullRes_full_summ

LV,pathway_prior
<chr>,<chr>
LV3,"Chromosome_Location_chr2p11, Chromosome_Location_chr14q32"
LV4,"BP_Positive Regulation of Type II Interferon Production (GO:0032729), GTEx_Tissues_Cells - EBV-transformed Lymphocytes Male 30-39 Up, GTEx_Tissues_Cells - EBV-transformed Lymphocytes Male 20-29 Up, GTEx_Tissues_Cells - EBV-transformed Lymphocytes Female 40-49 Up"
LV5,GTEx_Tissues_Muscle - Skeletal Male 60-69 Up
LV6,TF_Perturbations_CREM KO MOUSE GSE29593 CREEDSID GENE 2559 DOWN
LV8,"GTEx_Tissues_Colon - Transverse Male 30-39 Up, GTEx_Tissues_Colon - Transverse Male 20-29 Up, GTEx_Tissues_Colon - Transverse Male 40-49 Up, GTEx_Tissues_Colon - Transverse Female 30-39 Up"
LV9,"GTEx_Tissues_Uterus Female 20-29 Up, GTEx_Tissues_Colon - Sigmoid Male 70-79 Up"
LV12,"KEGG_Phagosome, GTEx_Tissues_Whole Blood Male 70-79 Up, TF_Perturbations_EZH2 SHRNA PROE HUMAN GSE59089 RNASEQ UP"
LV13,"Human_Gene_Atlas_SmoothMuscle, TF_Perturbations_NFKB1 ACTIVATION HUMAN GSE20736 CREEDSID GENE 2522 UP"
LV14,"GTEx_Tissues_Cells - Cultured Fibroblasts Male 40-49 Up, GTEx_Tissues_Artery - Aorta Female 70-79 Up"


In [11]:
gtex_tissues_res <- readRDS(file.path(output_data_dir, "gtex_GTEx_Tissues_V8_2023_PLIER2.rds"))

In [12]:
head(gtex_tissues_res$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0,0,0.0000000,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.2736694,0.0000000,0.0000000
RP11-34P13.15,0,0,0.0000000,0,0,0.3882859,0.1727005,0,0,0,⋯,0,0,0,0,0.3028549,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0.1069786,0,0,0.3685774,0.1644808,0,0,0,⋯,0,0,0,0,0.3337175,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0.0000000,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.2109335,0,0,0.1699839,0.0000000,0.0000000
AP006222.2,0,0,0.0000000,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.3321186,0,0,0.1503435,0.1533971,0.0000000
MTND1P23,0,0,0.0000000,0,0,0.0000000,0.0000000,0,0,0,⋯,0,0,0,0,0.4075553,0,0,0.0000000,0.0000000,0.4625319


In [8]:
head(gtex_fullRes_full$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.2729053,0.0000000,0.0000000
RP11-34P13.15,0,0,0,0,0,0.3858035,0,0,0,0,⋯,0,0,0,0,0.2888483,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0,0,0,0.3677623,0,0,0,0,⋯,0,0,0,0,0.3150313,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.2097148,0,0,0.1678578,0.0000000,0.0000000
AP006222.2,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.3023610,0,0,0.1620341,0.1618168,0.0000000
MTND1P23,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.4193483,0,0,0.0000000,0.0000000,0.4782187


In [15]:
head(gtex_tissues_res$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
LV1,0.05560544,-0.147110809,0.42570469,0.33897513,0.05234401,-0.12010717,0.09814678,-0.15140819,-0.099468776,-0.07921416,⋯,0.11262388,-0.10585147,0.07621861,-0.11323253,-0.049832321,0.19096230,-0.23235031,0.72098205,-0.18707945,0.058366720
LV2,0.02291564,-0.078594596,-0.04632225,0.06736278,0.02128422,0.07850724,0.13886374,0.04599792,0.075792184,0.04834272,⋯,-0.04320827,-0.08756574,-0.01638932,-0.04233717,-0.038304279,-0.02507263,-0.01900201,-0.09022222,-0.04884114,0.009400149
LV3,-0.01500113,-0.150867828,0.28939435,0.12219831,0.32802720,0.90143393,0.01859357,0.11338782,0.219631037,-0.13581845,⋯,0.02726810,-0.02318928,0.47449205,0.37046307,-0.008851576,0.04505481,0.01078102,-0.03956897,-0.13009395,0.169133346
LV4,-0.03263686,-0.006549777,-0.02828416,-0.03667213,-0.01644925,0.04404620,-0.03890090,-0.05991251,-0.054745158,-0.03492089,⋯,-0.02845289,-0.05809898,-0.03790735,-0.03157724,-0.056246184,-0.12779883,-0.04291276,-0.06881558,-0.01295887,-0.046540194
LV5,-0.31363565,-0.303828453,-0.43200202,-0.26349933,0.19215938,-0.26270178,-0.23280864,0.10248647,-0.208296950,-0.15816556,⋯,0.60963022,0.24192089,0.12396011,0.17905923,0.024285903,-0.16712629,0.15555938,-0.30005080,0.09550728,-0.047114612
LV6,0.10595610,-0.019411424,0.08784759,0.04024979,-0.12792783,-0.09144512,0.09289529,-0.01793144,-0.001319782,-0.06876859,⋯,-0.13178110,-0.08554797,0.03816479,-0.11999538,-0.048772126,0.01832179,-0.14148136,-0.01820480,-0.11411937,-0.011693786


In [16]:
head(gtex_fullRes_full$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
LV1,0.002306252,-0.185842321,0.35002089,0.28008779,0.03741540,-0.17881736,0.05909193,-0.124934915,-0.134987474,-0.09942754,⋯,0.13669629,-0.06866715,0.15103172,-0.098867918,-0.04976707,0.240907960,-0.207479608,0.65970771,-0.210369586,0.012693192
LV2,-0.001847560,-0.075999344,-0.04737399,0.02304581,0.01536249,0.04331927,0.03372155,0.028448348,0.031978199,0.03312909,⋯,-0.03185956,-0.05057576,-0.02081007,-0.045028212,-0.03845664,-0.099335916,-0.003038646,-0.08989603,-0.060674435,-0.033551958
LV3,-0.027954407,-0.129948939,0.26582091,0.08924263,0.30244911,0.87474845,-0.02569031,0.078981500,0.226039505,-0.13948102,⋯,0.01776362,-0.03777878,0.45130405,0.307856381,-0.01438259,0.009519779,-0.012460737,-0.04186983,-0.113257773,0.168239593
LV4,-0.036832330,-0.007790803,-0.04204723,-0.04715276,-0.02483749,0.03229146,-0.04454135,-0.072593807,-0.071256375,-0.02608708,⋯,-0.03073360,-0.06369364,-0.04507193,-0.028531709,-0.06508061,-0.151495837,-0.049594277,-0.07414403,-0.007885464,-0.043519634
LV5,-0.203325387,0.162232776,-0.29083327,-0.13574123,-0.02583639,-0.12747865,-0.13261264,0.073610554,-0.133929679,-0.11232567,⋯,0.15623320,0.10487041,0.04305969,0.008240055,-0.08353332,-0.059114773,0.036619703,-0.18209372,0.398362266,-0.097705168
LV6,0.114689573,-0.023394086,0.09164771,0.03693860,-0.13041657,-0.08656338,0.08390764,-0.008838941,-0.000792831,-0.06567368,⋯,-0.13693597,-0.08687173,0.03974802,-0.120357638,-0.05736384,0.013140055,-0.127938466,-0.02662199,-0.123636464,-0.004993658


In [14]:
cor(as.vector(gtex_tissues_res$Z), as.vector(gtex_fullRes_full$Z), use = "complete.obs")

[1] 0.9463082

In [17]:
cor(as.vector(gtex_tissues_res$B), as.vector(gtex_fullRes_full$B), use = "complete.obs")

[1] 0.9597238